In [1]:
x = [1,2,3,56,775,88899,54,3,44,-1.5,-6,-54353234,0.545,1234.34576234]
def reverse(lista):
  rev = []
  for x in range(0,len(lista)):
    rev.append(lista.pop())
  return rev
reverse(x)

[1234.34576234, 0.545, -54353234, -6, -1.5, 44, 3, 54, 88899, 775, 56, 3, 2, 1]

In [3]:
!pip install py-sudoku
from sudoku import Sudoku

puzzle = Sudoku(3,seed=3).difficulty(0.8)
puzzle.board


def transform_sudoku(board):
  sup = []
  for linha in puzzle.board:
    sup.append([str(x) if x != None else ' ' for x in linha])
  return sup
a = transform_sudoku(puzzle.board)
a

[[' ', '2', ' ', ' ', ' ', ' ', ' ', ' ', '5'],
 [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
 [' ', '1', ' ', ' ', ' ', ' ', '7', ' ', ' '],
 [' ', ' ', '4', ' ', ' ', ' ', '6', ' ', ' '],
 [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
 [' ', ' ', ' ', ' ', '5', '3', ' ', ' ', ' '],
 [' ', ' ', ' ', ' ', ' ', ' ', '5', ' ', ' '],
 [' ', ' ', ' ', '9', ' ', ' ', '4', '7', ' '],
 [' ', '6', ' ', '4', ' ', '8', '3', ' ', '2']]

In [4]:
solver(a)

NameError: name 'solver' is not defined

In [5]:
import pandas as pd
import numpy as np
from copy import deepcopy


import sys
path = 'drive/MyDrive/Colab Notebooks/Projetos/Sudoku/Desenvolvimento'
sys.path.append(path)
import sudoku_levels as sudoku_lv


In [6]:
def insert_number(matriz,linha,coluna,numero):
  sup = deepcopy(matriz)
  sup[linha][coluna] = numero
  return sup

In [7]:
def create_sudoku():
  sudoku = [[' ' for x in range(1,10)] for x in range(1,10)]
  for i in range(0,9):
      n = input()
      n = list(n)
      for j, x in enumerate(n):
        if str(x) not in [str(y) for y in range(1,10)]:
          n[j] = ' '
        sudoku = insert_number(sudoku,i,j,n[j])
  return sudoku

In [8]:
#Get Valores do Sudoku
def get_row(matriz, n):
  return matriz[n]

def get_column(matriz, n):
  coluna = []
  for linha in matriz:
    coluna.append(linha[n])
  return coluna

def get_square(matriz, n):
  square = []
  square_dict = {0:(0,0),1:(0,3),2:(0,6),
               3:(3,0),4:(3,3),5:(3,6),
               6:(6,0),7:(6,3),8:(6,6)}
  linha_base = square_dict[n][0]
  coluna_base = square_dict[n][1]
  for i in range(linha_base,linha_base+3,1):
    for j in range(coluna_base,coluna_base+3,1):
      square.append(matriz[i][j])
  return square

def get_square_number(linha,coluna):
  if linha <=2 and coluna <= 2:
    return 0
  elif linha <=2 and coluna <= 5:
    return 1
  elif linha <=2 and coluna <= 8:
    return 2
  elif linha <=5 and coluna <= 2:
    return 3
  elif linha <=5 and coluna <= 5:
    return 4
  elif linha <=5 and coluna <= 8:
    return 5
  elif linha <=8 and coluna <= 2:
    return 6
  elif linha <=8 and coluna <= 5:
    return 7
  elif linha <=8 and coluna <= 8:
    return 8

In [9]:
#Validações
def check_no_repeated(lista):
  if any(lista.count(x) > 1 for x in lista if x != ' '):
    return False
  else:
    return True

'''def check_no_empty(matriz):
  for linha in matriz:
    if '-' in linha:
      return False
  return True'''

def check_1to9(lista):
  if set([str(x) for x in range(1,10)]) == set(lista):
    return True
  else:
    return False

def check(matriz):
  #print('Validando Linhas...')
  for l in range(0,9):
    if check_no_repeated(get_row(matriz,l)) == False:
      return False
    if check_1to9(get_row(matriz,l)) == False:
      return False

  #print('Validando Colunas...')
  for c in range(0,9):
    if check_no_repeated(get_column(matriz, c)) == False:
      return False
    if check_1to9(get_column(matriz,c)) == False:
      return False

  #print('Validando Quadrados...')
  for s in range(0,9):
    if check_no_repeated(get_square(matriz, s)) == False:
      return False
    if check_1to9(get_square(matriz,s)) == False:
      return False

  print('Sudoku Completo!')
  return True

In [10]:
def missing_numbers(lista_linha,lista_coluna,lista_square):
  missing = [str(n) for n in range(1,10)]
  missing = [item for item in missing if item not in lista_linha]
  missing = [item for item in missing if item not in lista_coluna]
  missing = [item for item in missing if item not in lista_square]
  return missing

def create_pencil_matriz(matriz):
  sup_pencil = deepcopy(matriz)
  #display('sup pencil', sup_pencil)
  pencil_matriz = [[' ' for x in range(1,10)] for x in range(1,10)]
  for y,val_y in enumerate(pencil_matriz):
    for x, val_x in enumerate(range(0,9)):
      if sup_pencil[y][x] == ' ':
        pencil_matriz[y][x] = missing_numbers(get_row(sup_pencil,y),get_column(sup_pencil,x),get_square(sup_pencil,get_square_number(y,x)))
  return pencil_matriz

def insert_obvious(matriz):
  flag=0
  sup_obvious = deepcopy(matriz)
  pencil_matriz = create_pencil_matriz(sup_obvious)
  for i, linhas in enumerate(pencil_matriz):
    for j, colunas in enumerate(linhas):
      possibilidades = colunas
      if len(possibilidades) == 1 and possibilidades != ' ' and flag == 0:
        #print('Adding {} to Element {},{}'.format(possibilidades[0],i,j))
        sup_obvious = insert_number(sup_obvious, i,j,possibilidades[0])
        del possibilidades
        flag = 1
  return sup_obvious


def big_pencil_size(matriz):
  pencil_matriz = create_pencil_matriz(matriz)
  tamanhos = []
  for linhas in pencil_matriz:
    for valores in linhas:
      if valores != ' ':
        tamanhos.append(len(valores))
  print('MIN TAMANHOS:',min(tamanhos) )
  if min(tamanhos) >=2:
    return True
  else:
    return False



In [11]:
def add_pilha(pilha, matriz, linha, coluna, n):
  matriz[linha][coluna] = n
  empilha(pilha,matriz)
  return pilha



In [12]:
'''def insert_guess(state,matriz, pilha):
  pilha = empilha(pilha,matriz)
  pencil_matriz = create_pencil_matriz(matriz)
  for i, linhas in enumerate(pencil_matriz):
    for j, valores in enumerate(linhas):
      if len(valores) >= 2:
        guess = valores[state[-1]]
        matriz = insert_number(matriz,i,j,guess)
        pilha = empilha(pilha,matriz)
        print('Adding GUESS {} to Element {},{}'.format(guess,i,j))
        return pilha'''


"def insert_guess(state,matriz, pilha):\n  pilha = empilha(pilha,matriz)\n  pencil_matriz = create_pencil_matriz(matriz)\n  for i, linhas in enumerate(pencil_matriz):\n    for j, valores in enumerate(linhas):\n      if len(valores) >= 2:\n        guess = valores[state[-1]]\n        matriz = insert_number(matriz,i,j,guess)\n        pilha = empilha(pilha,matriz)\n        print('Adding GUESS {} to Element {},{}'.format(guess,i,j))\n        return pilha"

In [ ]:
#b = np.array(get_len_pencil(matriz_1))
#print(np.where(b == 2)[0][0])
#print(np.where(b == 2)[1][0])
#np.where(b == 6)[0].size

In [13]:
def get_len_pencil(matriz):
  pencil = create_pencil_matriz(matriz)
  size_pencil = [[len(x) if x != ' ' else 0 for x in lista] for lista in pencil]
  return size_pencil

def get_ij_minimun_pencil(matriz):
  b = np.array(get_len_pencil(matriz))
  for n in range(2,10):
    onde = np.where(b==n)
    if onde[0].size != 0:
      i = onde[0][0]
      j = onde[1][0]
      return i,j
  return -1,-1


In [14]:
def check_backtrack(matriz, guess_matriz,log=False):
  i,j = get_ij_minimun_pencil(matriz)
  if i == -1 and j == -1:
    return False
  if log:
    print('i,j para verificar no futuro: {}, {}'.format(i,j))
  valores = create_pencil_matriz(matriz)[i][j]
  valores = [x for x in valores if x not in guess_matriz[i][j]]
  if len(valores) == 0:
    return True
  else:
    return False

In [15]:
def insert_guess(matriz, guess_matriz,log=False):
  sup = deepcopy(matriz)
  guess_sup = deepcopy(guess_matriz)
  i,j = get_ij_minimun_pencil(sup)
  valores = create_pencil_matriz(sup)[i][j]
  if log:
    print('Trying to insert guess on {},{}'.format(i,j))
    print('Possiveis Valores: {}'.format(valores))
  sup,guess_sup = guess_maker(sup,valores,guess_sup,i,j)
  return i,j,sup,guess_sup


In [16]:
def guess_maker(matriz,valores,guess_matriz,i,j,log=False):
  sup_maker = deepcopy(matriz)
  guess_sup_maker = deepcopy(guess_matriz)
  valores = [x for x in valores if x not in guess_sup_maker[i][j]]

  guess_sup_maker[i][j].append(valores[0])
  guess = valores[0]
  sup_maker = insert_number(sup_maker,i,j,guess)
  if log:
    print('Adding GUESS {} to Element {},{}'.format(guess,i,j))
  return sup_maker, guess_sup_maker

In [17]:
def backtrack(matriz,guess_matriz,log=False):
  i,j = get_ij_minimun_pencil(matriz)
  print('Minimo i,j = {},{}'.format(i,j))
  valores = create_pencil_matriz(matriz)[i][j]
  valores = [x for x in valores if x not in guess_matriz[i][j]]
  if log:
    display('VALORES backtrack: ',valores)
  if len(valores) == 0:
    return True
  else:
    return False

In [18]:
def save_guess(state,matriz, linha,coluna,guess):
  state.append({'matriz':matriz, 'linha':linha,'coluna':coluna,'last_guess':guess})
  return state
def areSame(A,B):
   for i in range(0,9):
      for j in range(0,9):
         if (A[i][j] != B[i][j]):
            return 0
   return 1

In [19]:
def add_obvios(matriz):
  sup = deepcopy(matriz)
  matriz = insert_obvious(matriz)
  while sup != matriz:
    sup = deepcopy(matriz)
    matriz = insert_obvious(sup)
  return matriz

In [20]:
'''sudoku = deepcopy(pilha[-1])
  temp_sudoku = deepcopy(sudoku)
  sudoku = insert_obvious(sudoku)
  while temp_sudoku != sudoku:
    temp_sudoku = deepcopy(sudoku)
    sudoku = insert_obvious(sudoku)
  print('Obvios Adicionados.')
  if check(sudoku) or stop_counter '''

"sudoku = deepcopy(pilha[-1])\n  temp_sudoku = deepcopy(sudoku)\n  sudoku = insert_obvious(sudoku)\n  while temp_sudoku != sudoku:\n    temp_sudoku = deepcopy(sudoku)\n    sudoku = insert_obvious(sudoku)\n  print('Obvios Adicionados.')\n  if check(sudoku) or stop_counter "

In [21]:
def inicia_sudoku(matriz):
  sudoku = deepcopy(matriz)
  guess_sudoku = [[[] for i in range(9)] for i in range(9)]
  pilha = []
  pilha_guess = []
  pilha = empilha(pilha,sudoku) #pilha[-1] contem o mais correto
  pilha_guess = empilha(pilha_guess,guess_sudoku)
  flag=1
  return sudoku,pilha, guess_sudoku, pilha_guess, flag

In [22]:
def count_guess_vazios(guess_sudoku):
  count = 0
  for i in guess_sudoku:
    for j in i:
      if len(j) == 0:
        count = count+1
  return count

In [23]:
def empilha(pilha, matriz):
  sup = deepcopy(pilha)
  matriz_sup = deepcopy(matriz)
  sup.append(matriz_sup)
  return sup

def desempilha(pilha):
  sup = deepcopy(pilha)
  sup.pop()
  return sup

In [24]:
def small_pencil_size(matriz):
  pencil_matriz = deepcopy(create_pencil_matriz(matriz))
  tamanhos = []
  for linhas in pencil_matriz:
    for valores in linhas:
      if valores != ' ':
        tamanhos.append(len(valores))
  if tamanhos == []:
    return True
  if min(tamanhos) == 0:
    return True
  else:
    return False

In [25]:
def guess_limite(pilha_guess):
  ultimo = deepcopy(pilha_guess[-1])
  anterior = deepcopy(pilha_guess[-2])
#verifica se esse guess é limitado (ultimo valor possivel) > comparar o -1 com o -2 para buscar o ultimo i.j do guess



In [26]:
def solver(new_sudoku,log=False):
  inicio = datetime.datetime.now()
  sudoku,pilha, guess_sudoku, pilha_guess,flag = inicia_sudoku(new_sudoku)
  stop_counter = 1
  max_pilha = 0
  display('Sudoku Inicial: ', sudoku)
  while flag == 1 and stop_counter <= 200:
    sudoku = deepcopy(pilha[-1]) #Sudoku mais atual
    guess_sudoku = deepcopy(pilha_guess[-1]) #Guess mais atual
    if log:
      print('Iteracao', stop_counter)
      display('Sudoku Inicio Iteracao:',sudoku)
    sudoku = add_obvios(sudoku) #Adiciona obvios
    if sudoku != pilha[-1]:
      if log:
        print('Obvios Adicionados.')
    if check(sudoku): #Fecha se Sudoku estiver terminado depois do obvio
      flag = 0
      display(sudoku)
      pilha = empilha(pilha,sudoku)
      break

    i,j,sudoku,guess_sudoku = insert_guess(sudoku,guess_sudoku) #Insere um guess

    pilha = empilha(pilha,sudoku) #Empilha Sudoku e Guess
    pilha_guess = empilha(pilha_guess,guess_sudoku)

    #Erro tipo 1 (Trocar de chute)
    if small_pencil_size(pilha[-1]):
      if log:
        print('Impossivel de continuar. Voltando 1 Sudoku.')
      pilha = desempilha(pilha)

    futuro_estimado = deepcopy(pilha[-1])
    futuro_estimado = add_obvios(futuro_estimado)
    if check(futuro_estimado): #Fecha se Sudoku estiver terminado depois do obvio
      flag = 0
      display(futuro_estimado)
      pilha = empilha(pilha,futuro_estimado)

      break

    #Se removermos 1 guess, precisamos remover todos os guess que ja estavam em seu limite
    while small_pencil_size(futuro_estimado) == True and ~check(sudoku): #Se for verificado que ainda nao terminou E com o proximo obvio vai dar pau
      if log:
        print('Impossivel de continuar. Voltando 1 Sudoku.')
      pilha = desempilha(pilha) #Desempilhar
      futuro_estimado = deepcopy(pilha[-1])
      futuro_estimado = add_obvios(futuro_estimado) #E verificar como o procimo proximo obvio vai ficar.
      while check_backtrack(futuro_estimado, pilha_guess[-1]): #Se não tiver mais valores possiveis no proximo local com o guess atual. trocar esse with por um while!
        pilha = desempilha(pilha)
        vazios_inicial = count_guess_vazios(pilha_guess[-1])
        pilha_guess = desempilha(pilha_guess)
        while(vazios_inicial == count_guess_vazios(pilha_guess[-1])):
          pilha_guess = desempilha(pilha_guess)
        futuro_estimado = deepcopy(pilha[-1])
        futuro_estimado = add_obvios(futuro_estimado)

    stop_counter = stop_counter +1
    if max_pilha < len(pilha):
      max_pilha = len(pilha)
    if log:
      display('Guess Pós iteracao:',pilha_guess[-1])
      print('Comprimento da pilha Sudoku = {}'.format(len(pilha)))
      print('-----------------------------------------------------------------------------------',stop_counter)
  fim = datetime.datetime.now()
  tempo_total = fim - inicio
  print(f'Tamanho maximo da pilha: {max_pilha}')
  print(f'Total de iteracoes necessarias: {stop_counter}')
  print(f'Tempo Total de Execucao: {str(tempo_total)[5:10]} segundos!')
  return pilha[-1],max_pilha,stop_counter,tempo_total

In [27]:
import datetime
solver(sudoku_lv.matriz_3,log=False)


'Sudoku Inicial: '

[['9', ' ', '4', ' ', ' ', ' ', '3', ' ', '1'],
 [' ', ' ', '7', '8', '3', '1', '4', ' ', ' '],
 [' ', ' ', ' ', ' ', ' ', '9', '2', '8', ' '],
 ['3', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
 ['4', ' ', ' ', '7', ' ', ' ', '8', ' ', ' '],
 [' ', '6', ' ', '9', '2', ' ', ' ', ' ', ' '],
 [' ', ' ', '2', ' ', '5', '7', '9', ' ', ' '],
 [' ', ' ', '5', ' ', ' ', ' ', ' ', '2', ' '],
 [' ', ' ', ' ', '2', '8', ' ', ' ', '7', ' ']]

Sudoku Completo!


[['9', '8', '4', '6', '7', '2', '3', '5', '1'],
 ['2', '5', '7', '8', '3', '1', '4', '6', '9'],
 ['6', '1', '3', '5', '4', '9', '2', '8', '7'],
 ['3', '7', '8', '4', '1', '5', '6', '9', '2'],
 ['4', '2', '9', '7', '6', '3', '8', '1', '5'],
 ['5', '6', '1', '9', '2', '8', '7', '3', '4'],
 ['8', '3', '2', '1', '5', '7', '9', '4', '6'],
 ['7', '4', '5', '3', '9', '6', '1', '2', '8'],
 ['1', '9', '6', '2', '8', '4', '5', '7', '3']]

Tamanho maximo da pilha: 8
Total de iteracoes necessarias: 28
Tempo Total de Execucao: 00.45 segundos!


([['9', '8', '4', '6', '7', '2', '3', '5', '1'],
  ['2', '5', '7', '8', '3', '1', '4', '6', '9'],
  ['6', '1', '3', '5', '4', '9', '2', '8', '7'],
  ['3', '7', '8', '4', '1', '5', '6', '9', '2'],
  ['4', '2', '9', '7', '6', '3', '8', '1', '5'],
  ['5', '6', '1', '9', '2', '8', '7', '3', '4'],
  ['8', '3', '2', '1', '5', '7', '9', '4', '6'],
  ['7', '4', '5', '3', '9', '6', '1', '2', '8'],
  ['1', '9', '6', '2', '8', '4', '5', '7', '3']],
 8,
 28,
 datetime.timedelta(microseconds=454305))

In [28]:
sudoku,pilha, guess_sudoku, pilha_guess,flag = inicia_sudoku(matriz_4)
stop_counter = 1

while flag == 1 and stop_counter <= 200:
  print('Iteracao', stop_counter)
  sudoku = deepcopy(pilha[-1]) #Sudoku mais atual
  guess_sudoku = deepcopy(pilha_guess[-1]) #Guess mais atual
  display('Sudoku Inicio Iteracao:',sudoku)
  sudoku = add_obvios(sudoku) #Adiciona obvios
  if sudoku != pilha[-1]:
    print('Obvios Adicionados.')
  if check(sudoku): #Fecha se Sudoku estiver terminado depois do obvio
    flag = 0
    display(sudoku)
    pilha = empilha(pilha,sudoku)
    break

  i,j,sudoku,guess_sudoku = insert_guess(sudoku,guess_sudoku) #Insere um guess

  pilha = empilha(pilha,sudoku) #Empilha Sudoku e Guess
  pilha_guess = empilha(pilha_guess,guess_sudoku)

  #Erro tipo 1 (Trocar de chute)
  if small_pencil_size(pilha[-1]):
    print('Impossivel de continuar. Voltando 1 Sudoku.')
    pilha = desempilha(pilha)

  futuro_estimado = deepcopy(pilha[-1])
  futuro_estimado = add_obvios(futuro_estimado)
  if check(futuro_estimado): #Fecha se Sudoku estiver terminado depois do obvio
    flag = 0
    display(futuro_estimado)
    pilha = empilha(pilha,futuro_estimado)
    break

  #Se removermos 1 guess, precisamos remover todos os guess que ja estavam em seu limite
  while small_pencil_size(futuro_estimado) == True and ~check(sudoku): #Se for verificado que ainda nao terminou E com o proximo obvio vai dar pau
    print('Impossivel de continuar. Voltando 1 Sudoku.')
    pilha = desempilha(pilha) #Desempilhar
    futuro_estimado = deepcopy(pilha[-1])
    futuro_estimado = add_obvios(futuro_estimado) #E verificar como o procimo proximo obvio vai ficar.
    while check_backtrack(futuro_estimado, pilha_guess[-1]): #Se não tiver mais valores possiveis no proximo local com o guess atual. trocar esse with por um while!
      pilha = desempilha(pilha)
      vazios_inicial = count_guess_vazios(pilha_guess[-1])
      pilha_guess = desempilha(pilha_guess)
      while(vazios_inicial == count_guess_vazios(pilha_guess[-1])):
        pilha_guess = desempilha(pilha_guess)
      futuro_estimado = deepcopy(pilha[-1])
      futuro_estimado = add_obvios(futuro_estimado)

  display('Guess Pós iteracao:',pilha_guess[-1])
  stop_counter = stop_counter +1
  print('Comprimento da pilha Sudoku = {}'.format(len(pilha)))
  print('------------------------------------------------------------------------------------------------------------------------------------------------------',stop_counter)

NameError: name 'matriz_4' is not defined

In [29]:
######BACKUP COMPLETO COM PRINTS VALIDACAO
sudoku,pilha, guess_sudoku, pilha_guess,flag = inicia_sudoku(matriz_2)
stop_counter = 1

while flag == 1 and stop_counter <= 200:
  print('Iteracao', stop_counter)
  sudoku = deepcopy(pilha[-1]) #Sudoku mais atual
  guess_sudoku = deepcopy(pilha_guess[-1]) #Guess mais atual
  display('Sudoku Inicio Iteracao:',sudoku)
  sudoku = add_obvios(sudoku) #Adiciona obvios
  if sudoku != pilha[-1]:
    print('Obvios Adicionados.')
  if check(sudoku): #Fecha se Sudoku estiver terminado depois do obvio
    flag = 0
    display(sudoku)
    break

  i,j,sudoku,guess_sudoku = insert_guess(sudoku,guess_sudoku) #Insere um guess

  #display('Sudoku Pós Chute:',sudoku)
  #display('Guess Pós Chute:',guess_sudoku)
  pilha = empilha(pilha,sudoku) #Empilha Sudoku e Guess
  pilha_guess = empilha(pilha_guess,guess_sudoku)
  if stop_counter >= 43:
    print('#######################################')
    display(pilha[-1])
    print('#######################################')
  #Erro tipo 1 (Trocar de chute)
  if small_pencil_size(pilha[-1]):
    print('Impossivel de continuar. Voltando 1 Sudoku.')
    pilha = desempilha(pilha)

  futuro_estimado = deepcopy(pilha[-1])
  futuro_estimado = add_obvios(futuro_estimado)
  if check(futuro_estimado): #Fecha se Sudoku estiver terminado depois do obvio
    flag = 0
    display(futuro_estimado)
    break
  if stop_counter >= 43:
    print('#######################################')
    display('futuro estimado:',futuro_estimado)
    print('#######################################')
  #display('futuro estimado:',futuro_estimado)
  print('small_pencil_size(futuro_estimado):',small_pencil_size(futuro_estimado))
  #Se removermos 1 guess, precisamos remover todos os guess que ja estavam em seu limite
  while small_pencil_size(futuro_estimado) == True and ~check(sudoku): #Se for verificado que ainda nao terminou E com o proximo obvio vai dar pau
    print('Impossivel de continuar. Voltando 1 Sudoku.')
    pilha = desempilha(pilha) #Desempilhar
    futuro_estimado = deepcopy(pilha[-1])
    futuro_estimado = add_obvios(futuro_estimado) #E verificar como o procimo proximo obvio vai ficar.
    while check_backtrack(futuro_estimado, pilha_guess[-1]): #Se não tiver mais valores possiveis no proximo local com o guess atual. trocar esse with por um while!
      print('ARGH')
      pilha = desempilha(pilha)
      #display('Guess no ARGH:',pilha_guess[-1])
      vazios_inicial = count_guess_vazios(pilha_guess[-1])
      pilha_guess = desempilha(pilha_guess)

      #pilha_guess, -
      while(vazios_inicial == count_guess_vazios(pilha_guess[-1])):
        pilha_guess = desempilha(pilha_guess)
      futuro_estimado = deepcopy(pilha[-1])
      futuro_estimado = add_obvios(futuro_estimado)
      #while():#verifica se o guess anterior é limitado (ultimo valor possivel) > comparar o -1 com o -2 para buscar o ultimo i.j do guess. Esse while abraca o de cima na verdade
      #  pilha_guess = desempilha(pilha_guess)

      display('Guess pos ARGH:',pilha_guess[-1])
  display('Sudoku Pós iteracao:',pilha[-1])
  display('Guess Pós iteracao:',pilha_guess[-1])
  stop_counter = stop_counter +1
  print('Comprimento da pilha Sudoku = {}'.format(len(pilha)))
  print('------------------------------------------------------------------------------------------------------------------------------------------------------',stop_counter)
######

NameError: name 'matriz_2' is not defined

In [ ]:
#Backup sem while de desempihal
sudoku,pilha, guess_sudoku, pilha_guess,flag = inicia_sudoku(matriz_2)
stop_counter = 1
while flag == 1 and stop_counter <= 40:
  print('Iteracao', stop_counter)
  sudoku = deepcopy(pilha[-1])
  guess_sudoku = deepcopy(pilha_guess[-1])
  display('Sudoku Inicio Iteracao:',sudoku)
  sudoku = add_obvios(sudoku)
  if sudoku != pilha[-1]:
    print('Obvios Adicionados.')
  if check(sudoku):
    flag = 0
    display(sudoku)
    break
  sudoku,guess_sudoku = insert_guess(sudoku,guess_sudoku)
  display('Sudoku Pós Chute:',sudoku)
  display('Guess Pós Chute:',guess_sudoku)
  pilha = empilha(pilha,sudoku)
  pilha_guess = empilha(pilha_guess,guess_sudoku)

  #Erro tipo 1 (Trocar de chute)
  if small_pencil_size(pilha[-1]):
    print('Impossivel de continuar. Voltando 1 Sudoku.')
    pilha = desempilha(pilha)

  futuro_estimado = deepcopy(pilha[-1])
  futuro_estimado = add_obvios(futuro_estimado)
  display('futuro estimado:',futuro_estimado)
  if check_backtrack(futuro_estimado, pilha_guess[-1]):
    print('ARGH')
    pilha = desempilha(pilha)
    display('Guess no ARGH:',pilha_guess[-1])
    vazios_inicial = count_guess_vazios(pilha_guess[-1])
    pilha_guess = desempilha(pilha_guess)
    while(vazios_inicial == count_guess_vazios(pilha_guess[-1])):
      pilha_guess = desempilha(pilha_guess)
    display('Guess pos ARGH:',pilha_guess[-1])

  '''  if backtrack(sudoku,guess_sudoku):
    print('YEEEEEEEEY')
    vazios_inicial = count_guess_vazios(guess_sudoku)
    print('Vazios: {}'.format(vazios_inicial))
    while(vazios_inicial == count_guess_vazios(pilha_guess[-1])):
      print('Desempilhou GUESS')
      pilha_guess = desempilha(pilha_guess)
      print(len(pilha_guess))
      display('pos troca guess',pilha_guess[-1])
      print(count_guess_vazios(pilha_guess[-1]))

    display('pos troca Sudoku',pilha[-1])
    display('pos troca guess',pilha_guess[-1])

  sudoku,guess_sudoku = insert_guess(sudoku,guess_sudoku)
  pilha = empilha(pilha,sudoku)
  pilha_guess = empilha(pilha_guess,guess_sudoku)
  print(len(pilha_guess))
  display('Sudoku', pilha[-1])
  display('Guess Sudoku', pilha_guess)
  if small_pencil_size(pilha[-1]):
    print('CACAAAAAAAAAAAA')
    pilha = desempilha(pilha)
    display('Sudoku desempilhado', pilha[-1])
    #display(pilha[-1])
  #if backtrack(pilha[-1],pilha_guess[-1]):
  #  print('DEU RUIM!')
  #  desempilha(pilha)
  #  desempilha(guess_sudoku)'''


  stop_counter = stop_counter +1
  print('--------------------------------------------------')

In [ ]:
#BACKUP!

sudoku = deepcopy(matriz_2)
guess_sudoku = [[[] for i in range(9)] for i in range(9)]
pilha = []
pilha_guess = []
pilha = empilha(pilha,sudoku) #pilha[-1] contem o mais correto
pilha_guess = empilha(pilha_guess,guess_sudoku)

flag = 1
stop_counter = 1
while flag == 1:
  print('Iteracao', stop_counter)
  sudoku = deepcopy(pilha[-1])
  sudoku = add_obvios(sudoku)
  print('Obvios Adicionados.')
  if check(sudoku) or stop_counter == 15:
    flag = 0
    display(sudoku)
    break
  if backtrack(sudoku,guess_sudoku):
    print('YEEEEEEEEY')
    vazios_inicial = count_guess_vazios(guess_sudoku)
    print('Vazios: {}'.format(vazios_inicial))
    while(vazios_inicial == count_guess_vazios(pilha_guess[-1])):
      print('Desempilhou GUESS')
      pilha_guess = desempilha(pilha_guess)
      print(len(pilha_guess))
      display('pos troca guess',pilha_guess[-1])
      print(count_guess_vazios(pilha_guess[-1]))

    display('pos troca Sudoku',pilha[-1])
    display('pos troca guess',pilha_guess[-1])

  sudoku,guess_sudoku = insert_guess(sudoku,guess_sudoku)
  pilha = empilha(pilha,sudoku)
  pilha_guess = empilha(pilha_guess,guess_sudoku)
  print(len(pilha_guess))
  display('Sudoku', pilha[-1])
  display('Guess Sudoku', pilha_guess)
  if small_pencil_size(pilha[-1]):
    print('CACAAAAAAAAAAAA')
    pilha = desempilha(pilha)
    display('Sudoku desempilhado', pilha[-1])
    #display(pilha[-1])
  #if backtrack(pilha[-1],pilha_guess[-1]):
  #  print('DEU RUIM!')
  #  desempilha(pilha)
  #  desempilha(guess_sudoku)

  print('--------------------------------------------------')
  stop_counter = stop_counter +1

In [ ]:
flag = 1
display(matriz_1)
pilha = []
pilha = empilha(pilha,matriz_1)
state = []
counter = 0
while (flag==1):
  matriz = pilha[-1].copy()
  matriz = insert_obvious(matriz)
  if check(matriz) == True:
    break
    flag = 0
  if areSame(matriz, pilha[-1]):
    display(matriz)
    display(pilha[-1])
    print('Nada Adicionado')
  display(matriz)
  #print(big_pencil_size(create_pencil_matriz(matriz)))
  #display(create_pencil_matriz(matriz))
  if big_pencil_size(create_pencil_matriz(matriz)) == True:
    print('Validando possibilidades')
    print('STATE antes: ',state)
    state.append(-1)
    state[-1] = state[-1] + 1
    print('STATE depois: ',state)

    pilha = insert_guess(state,matriz, pilha)


  elif small_pencil_size(create_pencil_matriz(matriz)) == True:
    print('XIIIIIIIII')
    #tag[-1] = tag[-1] + 1
    #desempilha
    #state.append('')


  if counter == 5:
    break
  counter = counter+1


In [ ]:
flag = 1
display(matriz)
pilha = []
pilha = empilha(pilha,matriz)
state = [0]
counter = 0
while (flag==1):
  matriz = pilha[-1]
  matriz = insert_obvious(matriz)
  #display(matriz)
  print(big_pencil_size(create_pencil_matriz(matriz)))
  display(create_pencil_matriz(matriz))
  if big_pencil_size(create_pencil_matriz(matriz)) == True:
    print('aaaaaa')
    pilha = insert_guess(state,matriz, pilha)

  elif small_pencil_size(create_pencil_matriz(matriz)) == True:
    print('a')
    tag[-1] = tag[-1] + 1
    desempilha
    state.append('')
  if check(matriz) == True:
    flag = 0
  if counter == 10:
    break
  counter = counter+1


In [ ]:
 matriz

[['5', '2', '6', '3', '1', '7', '4', '9', '9'],
 ['3', '7', '4', '8', '9', '2', '6', '1', '5'],
 ['1', '9', '9', '4', '5', '6', '7', '8', '2'],
 ['2', '4', '5', '9', '3', '8', '1', '7', '6'],
 [' ', '6', ' ', '2', ' ', ' ', '3', '5', '8'],
 ['9', ' ', ' ', ' ', '7', ' ', '2', '2', '4'],
 [' ', '5', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
 [' ', '1', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
 [' ', ' ', '8', '1', ' ', '9', '5', '4', ' ']]

In [ ]:
pilha = []
pilha.append(matriz)
pilha.pop()
pilha

#teste = create_sudoku()
def insert_guess(matriz, guess_matriz):
  pencil_matriz = create_pencil_matriz(matriz)
  for i, linhas in enumerate(pencil_matriz):
    for j, valores in enumerate(linhas):
      print('size valores: ',len(valores))
      if len(valores) == 2:
        matriz = guess_maker(matriz,valores,guess_matriz,i,j)
      elif len(valores) == 3:
        matriz = guess_maker(matriz,valores,guess_matriz,i,j)
      elif len(valores) == 4:
        matriz = guess_maker(matriz,valores,guess_matriz,i,j)
      elif len(valores) == 5:
        matriz = guess_maker(matriz,valores,guess_matriz,i,j)
      elif len(valores) == 6:
        matriz = guess_maker(matriz,valores,guess_matriz,i,j)
      elif len(valores) == 7:
        matriz = guess_maker(matriz,valores,guess_matriz,i,j)
      elif len(valores) == 8:
        matriz = guess_maker(matriz,valores,guess_matriz,i,j)
      elif len(valores) == 9:
        matriz = guess_maker(matriz,valores,guess_matriz,i,j)
    return matriz